In [ ]:
from model import mainModel
import config
from main import get_seqs
import torch
import numpy as np
import pandas as pd
import os
import dgl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = dgl.DGLGraph()  # Example graph
model = mainModel(g)


model_path = 'best_model.pth'
net = mainModel(g, in_dims, config.embedding_size, config.num_layers, config.num_gnns,
                config.num_heads, config.dropout, temper=config.temper, num_type=len(node_cnt), beta=config.beta)
net.load_state_dict(torch.load(model_path))
net = net.to(device)
net.eval()

# Prepare data
new_dg_seq = get_seqs(g, new_drug_nodes).to(device)
new_pt_seq = get_seqs(g, new_protein_nodes).to(device)

# Make predictions
with torch.no_grad():
    predictions = net(new_features_list, new_dg_seq, new_pt_seq, type_emb, node_type).cpu().numpy()

# Save predictions
output_df = pd.DataFrame({
    'Drug': new_drugs,
    'Protein': new_proteins,
    'Prediction': predictions
})
output_df.to_csv('predictions.csv', index=False)


In [ ]:
import pandas as pd

# Load Pharos data (reclassified proteins)
pharos_data = pd.read_csv('pharos_reclassified.csv')

# Extract relevant information: drug, protein, and interaction (label)
drugs = pharos_data['drug_smiles']  # Example: SMILES strings for drugs
proteins = pharos_data['protein_sequence']  # Example: protein sequences
interactions = pharos_data['interaction_label']  # Binary interaction (0 or 1)

# Convert SMILES strings and protein sequences into numerical representations (embeddings)
# This would typically involve using an embedding model or descriptor generator
drug_features = convert_smiles_to_features(drugs)  # Function to convert SMILES to features
protein_features = convert_sequences_to_features(proteins)  # Function to convert sequences to features

# Construct the adjacency matrix based on the interactions (drug-protein pairs)
# Create a graph where nodes are drugs and proteins and edges represent interactions
adjM = create_adjacency_matrix(drugs, proteins, interactions)

# Convert adjacency matrix into a DGL graph
g = dgl.DGLGraph(adjM)
g = dgl.remove_self_loop(g)

# Generate node sequences for drug and protein nodes
dg_seq = get_seqs(g, list(range(len(drugs))))  # Sequences for drugs
pt_seq = get_seqs(g, list(range(len(drugs), len(drugs) + len(proteins))))  # Sequences for proteins

# Split the data into training and test sets
train_set, test_set = split_data(pharos_data)

# Pass the data into the model
train_model(
    net, optimizer, fold_nums, epoch, [drug_features, protein_features], node_type, type_emb, dg_seq, pt_seq, train_set[:, 2], model_path
)


In [23]:
%pip install torchaudio==2.1.0  torch==2.1.0 torchdata==0.7.0

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from model import mainModel
from config import Config
from main_dp import get_seqs
import torch
import numpy as np
import pandas as pd
import dgl

config = Config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example: Load a graph from an edge list
edges = torch.tensor([[0, 1], [1, 2], [2, 3]])  # Example edges
g = dgl.graph((edges[0], edges[1]))

# Add node features (128 dimensions as an example)
g.ndata['feat'] = torch.randn(g.num_nodes(), 128)
input_dimensions = g.ndata['feat'].shape[1]

# Load the model
model_path = 'best_model.pth'
net = mainModel(g, input_dimensions, config.embedding_size, config.num_layers, config.num_gnns,
                config.num_heads, config.dropout, temper=config.temper, num_type=4, beta=config.beta)
net.load_state_dict(torch.load(model_path, map_location=device))
net = net.to(device)
net.eval()

# Load new input data (example)
new_drug_nodes = torch.randint(0, g.num_nodes(), (10,)).to(device)  # Example indices
new_protein_nodes = torch.randint(0, g.num_nodes(), (10,)).to(device)

# Get sequences
new_dg_seq = get_seqs(g, new_drug_nodes).to(device)
new_pt_seq = get_seqs(g, new_protein_nodes).to(device)

# Example for type_emb and node_type
type_emb = torch.nn.Embedding(10, 128).to(device)
node_type = torch.randint(0, 10, (g.num_nodes(),)).to(device)

# Make predictions
with torch.no_grad():
    new_features_list = g.ndata['feat'][new_drug_nodes]
    predictions = net(new_features_list, new_dg_seq, new_pt_seq, type_emb, node_type).cpu().numpy()

# Save predictions
output_df = pd.DataFrame({
    'Drug': new_drug_nodes.cpu().numpy(),
    'Protein': new_protein_nodes.cpu().numpy(),
    'Prediction': predictions
})
output_df.to_csv('predictions.csv', index=False)


FileNotFoundError: Cannot find DGL C++ graphbolt library at /opt/anaconda3/envs/geometric/lib/python3.11/site-packages/dgl/graphbolt/libgraphbolt_pytorch_2.5.1.dylib

In [26]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_dict = torch.load('best_model.pth', map_location=device)
print(state_dict.keys())

odict_keys(['vars.fc_list_w_0', 'vars.fc_list_b_0', 'vars.fc_list_w_1', 'vars.fc_list_b_1', 'vars.proj_w_0_d', 'vars.proj_b_0_d', 'vars.proj_w_1_d', 'vars.proj_b_1_d', 'vars.proj_w_0_p', 'vars.proj_b_0_p', 'vars.proj_w_1_p', 'vars.proj_b_1_p', 'vars.ml_fc_w1', 'vars.ml_fc_b1', 'vars.ml_fc_w2', 'vars.ml_fc_b2', 'vars.ml_fc_w3', 'vars.ml_fc_b3', 'vars.gcn_w_0', 'vars.gcn_w_1', 'vars.gcn_w_2', 'vars.GT_linear_l_w_dg_0', 'vars.GT_linear_r_w_dg_0', 'vars.GT_att_l_dg_0', 'vars.GT_att_r_dg_0', 'vars.linear_final_dg_0', 'vars.GT_linear_l_w_dg_1', 'vars.GT_linear_r_w_dg_1', 'vars.GT_att_l_dg_1', 'vars.GT_att_r_dg_1', 'vars.linear_final_dg_1', 'vars.GT_linear_l_w_pt_0', 'vars.GT_linear_r_w_pt_0', 'vars.GT_att_l_pt_0', 'vars.GT_att_r_pt_0', 'vars.linear_final_pt_0', 'vars.GT_linear_l_w_pt_1', 'vars.GT_linear_r_w_pt_1', 'vars.GT_att_l_pt_1', 'vars.GT_att_r_pt_1', 'vars.linear_final_pt_1', 'proj_d.vars.fc_list_w_0', 'proj_d.vars.fc_list_b_0', 'proj_d.vars.fc_list_w_1', 'proj_d.vars.fc_list_b_1', 'p

/var/folders/0v/h_hg_1mj17d5ssrjq1z2m6x80000gp/T/ipykernel_91377/4293087749.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('best_model.pth', map